## PS1: Optimize the feature set of a rack-mounted M2 MacPro Server
The [new MacPro with the M2 Ultra chip has been released](https://www.apple.com/shop/buy-mac/mac-pro/rack#). The MacPro M2 has several configuration options broadly organized into five categories: `{CPU, Memory, Storage, Accessories, Software}` with multiple options per category:
* The `CPU` category has `2` options. Only one option can be selected from the `CPU` category
* The `Memory` category has `3` options. Only one option can be selected from the `Memory` category.
* The `Storage` category has `4` options. Only one option can be selected from the `Storage` category.
* The `Accessories` category has `3` options. Only one option can be selected from the `Accessories` category
* The `Software` category has `2` options. Neither or both options can be selected from the `Software` category

### Problem statement
Using a `linear` utility model and budget and feature constraints, compute the optimal set of features for a rack-mounted MacPro M2. This problem will be similar to the examples discussed in the lecture and discussion, except for one crucial element: the decision variables (our choices) will be `binary`, i.e., $x_{i}\in{0,1}$ where a `0` indicates that we do NOT choose feature $i$. In contrast, a value of `1` indicates that we choose option $i$. 

Formally, an agent has a set of $n$ configuration options $X = \left\{x_{i}\right\}_{i=1}^{n}$, a `Linear` utility function, and a total of $I$ units of resource to allocate, e.g., money, and potentially other constraints. An optimal agent maximizes its utility subject to its resource and other constraints:

\begin{eqnarray}
\text{maximize}~\mathcal{O} &=& \sum_{i\in{1,\dotsc,n}}\alpha_{i}x_{i} \\
\text{subject to}~\sum_{i\in{1,\dotsc,n}}c_{i}x_{i} & = & I\\
\text{and}~\mathbf{C}\mathbf{x} & \leq & \mathbf{b} \\
\text{and}~x_{i}&\in&{0,1}\qquad{i=1,2,\dots,n}
\end{eqnarray}

The quantity $c_{i}\geq{0}~\forall{i}$ denotes the cost of option $i$, $\alpha_{i}$ denotes user-specified coefficient in the `Linear` utility function, $x_{i}\in{0,1}$ represents the choice of option $i$, and $\mathbf{C}\mathbf{x} \leq \mathbf{b}$ represents additional constraints governing the decision.

#### List of Tasks
* __Task 1__: Specify $\alpha$- and $c$-vectors for the problem
* __Task 2__: Specify the additional constraint matrix $\mathbf{C}$
* __Task 3__: Specify the problem object and solve the problem
* __Task 4__: Try at least two different weighting schemes and different budget values, and explore how these design choices influence the optimal configuration (one of these __can__ be the `default` values specified below).

### Setup
The computations in this problem set rely on [VLDecisionsPackage.jl](https://github.com/varnerlab/VLDecisionsPackage.jl.git) and several external `Julia` packages. To load the required packages and any custom codes the teaching team has developed to work with these packages, we [include](https://docs.julialang.org/en/v1/manual/code-loading/) the `Include.jl` file):

In [12]:
include("Include.jl");

  Activating project at `C:\Users\lwdes\CHEME 5760\Lauren_deSilva_CHEME5760_FinalProject`
    Updating registry at `C:\Users\lwdes\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
  No Changes to `C:\Users\lwdes\CHEME 5760\Lauren_deSilva_CHEME5760_FinalProject\Project.toml`
  No Changes to `C:\Users\lwdes\CHEME 5760\Lauren_deSilva_CHEME5760_FinalProject\Manifest.toml`


#### Specify constants and other static stuff

In [13]:
number_of_choices = 14;
bounds_array = Array{Float64,2}(undef, number_of_choices,2)
for i ∈ 1:number_of_choices
    bounds_array[i,1] = 0.0
    bounds_array[i,2] = 1.0;
end

*Note: Bounds are between 0 and 1 because this is a binary allocation problem where we only assign either 0 or 1 to a solution.* The built in binary fucntion automatically identifies the bounds as a discrete problem. So even thoguh we have 0.0 to 1.0, we will only consider 0 or 1 not every conintuous number in between.

## Task 1: Specify the configuration array
The `configuration_array` is a `14` $\times$ `2` array holding perception and cost information about the problem. Each row of the `configuration_array` contains data for a particular MacPro configuration option. The first column contains the coefficients of the `Linear` utility function, i.e., the elements of the $\alpha$-vector, while the unit price of the features, i.e., elements of the $c$-vector, are in the second column.  

#### Feature prices
The prices of each configuration option have been estimated from the [Apple MacPro server website](https://www.apple.com/shop/buy-mac/mac-pro/rack#).

#### Linear utility weighting scheme
In `PS1`, we'll use a category-based weighting scheme. In each of the five categories, allocate `1` unit of weight:

* In each category, the coefficients in the `Linear` utility function for options in that category must sum to one.

For example, if you have equal feelings about three options in a category, then `0.33, 0.33, 0.33` would be a typical scheme. On the other hand, if you are excited about feature `1` over the other two in the category, then `0.8,0.1,0.1` could be an appropriate weight scheme.

In [14]:
configuration_array = [
    #The first column is the alpha vector,a, i.e how we feel about the feature
    #The second column is the unit price of each feature, c 
    # CPU options
    0.05 2640.0    ; # 1 CPU 1
    0.95 3649.0    ; # 2 CPU 2

    # Memory options
    0.05 3840.0  ; # 3 Memory 1
    0.05 4640.0  ; # 4 Memory 2
    0.9 3600.0  ; # 5 Memory 3

    # Storage options -
    0.05 1440.0   ; # 6 Storage 1
    0.05 1840.0   ; # 7 Storage 2
    0.05 2440.0   ; # 8 Storage 3
    0.85 3640.0   ; # 9 Storage 4

    # Accessory options
    0.05 79.0    ; # 10 Accessory 1
    0.05 129.0   ; # 11 Accessory 2
    0.90 149.0   ; # 12 Accessory 3
    
    # Software options
    0.05 299.0     ; # 13 Software 1
    0.95 149.0     ; # 14 Software 2
];

### Task 2: Specify the feature constraint matrix `C`
In each category, only a finite number of options can be selected simultaneously, typically only a single option, with the exception being the `Software` category, which is unconstrained (can have from zero up to two items selected). Because the decision variables are binary, we can implement this requirement with an additional set of constraints of the form:

$$
\begin{equation}
\mathbf{C}\mathbf{x} = \mathbf{1}
\end{equation}
$$

where $\mathbf{C}$ denotes the constraint matrix, $\mathbf{x}$ denotes the choice vector and $\mathbf{1}$ denotes a vector of `1`'s. 
Specify the $\mathbf{C}$ matrix for this problem:

In [15]:
C = zeros(4,14); # understanding qestion: why is this a 4 x 14 array?

!Answer!
- This is a 4 x 14 matrix aimed at ensuring the choice vector is constraied. There are 4 rows in this matrix, one row for each main category we are selcting from (CPU, Memory, Storage, Accessory). 
- There are 14 columns in this matrix, one column for each choice option in the array.

In [16]:
C[1,1]=1;
C[1,2]=1;
C[2,3]=1;
C[2,4]=1;
C[2,5]=1;
C[3,6]=1;
C[3,7]=1;
C[3,8]=1;
C[3,9]=1;
C[4,10]=1;
C[4,11]=1;
C[4,12]=1;

In [17]:
C

4×14 Matrix{Float64}:
 1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0

__Note__: Julia's Array syntax is similar to Matlab/Octave, except with square brackets. See [the Array documentation](https://docs.julialang.org/en/v1/base/arrays/) or various other [Julia tutorials on the web](https://www.tutorialspoint.com/julia/julia_arrays.htm) about working with the Array data structure.

### Task 3: Specify the problem object, and solve for the optimal configuration
Finally, build a `MySimpleBinaryVariableLinearChoiceProblem` model using the `build(...)` method, set this instance to the `problem` variable, and then pass the `problem` object to the `solve(...)` method. The `solve(...)` method will solve the `ILP` problem using the [GLPK.jl](https://github.com/jump-dev/GLPK.jl) interface to the [GLPK linear programming solver](https://www.gnu.org/software/glpk/).

In [18]:
α = configuration_array[:,1];
c = configuration_array[:,2];
I = 10000; # default budget is 10K USD

# Fill me in
problem= build(MySimpleBinaryVariableLinearChoiceProblem, (
    initial= 1*ones(14),
    α=α, #Coeffiecents
    c=c, #Price of each element
    I=I, #total budget
    bounds=bounds_array,
    C=C
        
));
solution=solve(problem);

In [19]:
solution

Dict{String, Any} with 3 entries:
  "argmax"          => [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, …
  "budget"          => 9686.0
  "objective_value" => 3.8

#### Check: Are the choice constraints enforced?
We can only select a fixed number of items from each category. Does your solution reflect this restriction?

In [9]:
# fill me in 
argmax_values = solution["argmax"]

14-element Vector{Float64}:
 0.0
 1.0
 0.0
 0.0
 1.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0

Yes, this option does satisfy the condtions. In each cateogry, the solution only gives one specific optimal choice. 

In [10]:
C*argmax_values

4-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0

### Task 4: How does changing the $\alpha$-vector (or the budget $I$) influence the configuration choice?

In [11]:
# Fill me in - this is an open ended question, so mess around with the alpha - values and I and jot down some observations. 

Things I noticed from changing the alpha parameters: 
- If I specifiy that I have a strong preference towards the cheapest option, my optimal choice is always the cheapest. 
- If I specify that I am indifferent and I weight all of the options with the same alpha value, my optiomal choice again is the one that saves the most money. 
- If I specify that I have a weak preference towards the most expensive option, my optiomal choice is one that splurges all all categories except storage. 
- If I specify that I have a strong preference towards the most expensive and I cut my budget down to $8,000, luxurious option, it says I cannot afford to purgchase any storage. 